In [ ]:
import torch
from exp.exp_main import Exp_Main
import numpy as np
from utils.tools import dotdict
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

import os
import datetime
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
%matplotlib inline

In [ ]:

# fix_seed = 2021
# np.random.seed(fix_seed)
# torch.manual_seed(fix_seed)
# np.random.seed(fix_seed)

In [ ]:

args = dotdict()
args.target = 'e_mu_current'
args.des = 'test'
args.dropout = 0.05
args.num_workers = 10
args.gpu = 0
args.lradj = 'type1'
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False

args.freq = 't'
args.checkpoints = './checkpoints/'
args.bucket_size = 4
args.n_hashes = 4
args.is_trainging = True
args.root_path = './datasets/'
args.data_path ='non_outliers_step=60_log_inf_train=0.9.csv' 
# args.data_path ='outliers_step=60_log_inf_train=0.9.csv' 
args.model_id='qber_96_12_60_loginf_t_non_outliers'
# args.model_id='qber_96_48_60_loginf_t_outliers'
args.model = 'Autoformer'
args.data = 'custom'
args.features = 'MS'
args.seq_len = 96
args.label_len = 12
args.pred_len = 1
args.e_layers = 6
args.d_layers = 8
args.n_heads = 8
args.factor = 3
args.enc_in = 6
args.dec_in =6
args.c_out = 6
args.d_model = 512
args.des = 'Exp'
args.itr = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 1
args.distil = False
args.output_attention = True
args.patience= 3
args.learning_rate = 0.0001
args.batch_size = 256
args.embed = 'timeF'
args.activation = 'gelu'
args.use_amp = False
args.loss = 'mse'
args.train_epochs = 40

exp = Exp_Main(args)

In [ ]:
torch.cuda.empty_cache()

In [ ]:

setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
    args.model_id,
    args.model,
    args.data,
    args.features,
    args.seq_len,
    args.label_len,
    args.pred_len,
    args.d_model,
    args.n_heads,
    args.e_layers,
    args.d_layers,
    args.d_ff,
    args.factor,
    args.embed,
    args.distil,
    args.des, 0)
print(setting)

In [ ]:
exp.train(setting)

In [ ]:
best_model_path=f'./checkpoints/{setting}/checkpoint.pth'

exp.model.load_state_dict(torch.load(best_model_path))

exp.test(setting)

In [ ]:
exp.args.root_path = './datasets/'
exp.args.data_path = 'data_step=60_test=0.1.csv'

prediction=exp.predict(setting, True)

In [ ]:
torch.cuda.empty_cache()


In [ ]:
setting='qber_96_48_120_loginf_t_non_outliers_Autoformer_custom_ftMS_sl96_ll48_pl1_dm512_nh8_el6_dl6_df2048_fc1_ebtimeF_dtFalse_Exp_0'
# setting='qber_96_48_120_loginf_t_outliers_Autoformer_custom_ftMS_sl96_ll48_pl1_dm512_nh8_el6_dl6_df2048_fc1_ebtimeF_dtFalse_Exp_0'


data_pred = np.load(f'./results/{setting}/pred.npy')

data_true = np.load(f'./results/{setting}/true.npy')



In [ ]:
start=0
end=1000
plt.figure(figsize=(20,5))
plt.plot(data_pred[start:end,0,0])
plt.plot(data_true[start:end,0,0])
plt.legend(['predicted','true'])
plt.show()

In [ ]:
start=8200
end=8700
plt.figure(figsize=(20,5))
plt.plot(data_pred[start:end,0,0])
plt.plot(data_true[start:end,0,0])
plt.legend(['predicted','true'])
plt.show()